In [96]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [97]:

chats = [
    (1, 1, 'damu', 'Qué es esto?'),
    (2, 2, 'martin', 'Un chat!'),
    (3, 1, 'damu', 'Ahhh! Y de donde salio? Whatsapp?'),
    (4, 2, 'martin', 'Sí! Cómo sabias?'),
    (5, 1, 'damu', 'Adivine'),
    (6, 3, 'luis', 'Hola!')
]

In [154]:
data = sc.parallelize(chats)

In [99]:
data.collect()

[(1, 1, 'damu', 'Qué es esto?'),
 (2, 2, 'martin', 'Un chat!'),
 (3, 1, 'damu', 'Ahhh! Y de donde salio? Whatsapp?'),
 (4, 2, 'martin', 'Sí! Cómo sabias?'),
 (5, 1, 'damu', 'Adivine'),
 (6, 3, 'luis', 'Hola!')]

In [100]:
data.count()

6

In [101]:
data.take(3)

[(1, 1, 'damu', 'Qué es esto?'),
 (2, 2, 'martin', 'Un chat!'),
 (3, 1, 'damu', 'Ahhh! Y de donde salio? Whatsapp?')]

In [102]:
def func(x):
    #print(x)
    #print(' x[0]: ', x[0])
    #print(' x[1]: \n ', x[1])
    return x[0]

data.takeOrdered(10,key = func)

[(1, 1, 'damu', 'Qué es esto?'),
 (2, 2, 'martin', 'Un chat!'),
 (3, 1, 'damu', 'Ahhh! Y de donde salio? Whatsapp?'),
 (4, 2, 'martin', 'Sí! Cómo sabias?'),
 (5, 1, 'damu', 'Adivine'),
 (6, 3, 'luis', 'Hola!')]

In [103]:
data.first()

(1, 1, 'damu', 'Qué es esto?')

In [155]:
data.countByKey()

AttributeError: 'collections.defaultdict' object has no attribute 'collect'

In [105]:
data.countByValue()

defaultdict(int,
            {(1, 1, 'damu', 'Qué es esto?'): 1,
             (2, 2, 'martin', 'Un chat!'): 1,
             (3, 1, 'damu', 'Ahhh! Y de donde salio? Whatsapp?'): 1,
             (4, 2, 'martin', 'Sí! Cómo sabias?'): 1,
             (5, 1, 'damu', 'Adivine'): 1,
             (6, 3, 'luis', 'Hola!'): 1})

In [106]:
data.reduce(lambda x,y: x+y)

(1,
 1,
 'damu',
 'Qué es esto?',
 2,
 2,
 'martin',
 'Un chat!',
 3,
 1,
 'damu',
 'Ahhh! Y de donde salio? Whatsapp?',
 4,
 2,
 'martin',
 'Sí! Cómo sabias?',
 5,
 1,
 'damu',
 'Adivine',
 6,
 3,
 'luis',
 'Hola!')

In [107]:
# El map, aplica a cada fila del rdd algo, usando ese mismo valor. Permite transformar.
data.map(lambda x: x[0]*x[0]).reduce(lambda x,y: x+y)

91

In [108]:
# basicamente es el que me permite aplicar condiciones
data.filter(lambda x: x[0]>3).collect()
# x es el dato completo
# x[0] es el primero
# x[1] ..

[(4, 2, 'martin', 'Sí! Cómo sabias?'),
 (5, 1, 'damu', 'Adivine'),
 (6, 3, 'luis', 'Hola!')]

In [109]:
data.map(lambda x: [a for a in range(0,x[0])]).collect()

[[0], [0, 1], [0, 1, 2], [0, 1, 2, 3], [0, 1, 2, 3, 4], [0, 1, 2, 3, 4, 5]]

In [110]:
# data.flatmap(func) esto serviria si lo de adentro devuelve una lista o algo asi

In [111]:
data.distinct().collect()

[(3, 1, 'damu', 'Ahhh! Y de donde salio? Whatsapp?'),
 (1, 1, 'damu', 'Qué es esto?'),
 (2, 2, 'martin', 'Un chat!'),
 (6, 3, 'luis', 'Hola!'),
 (5, 1, 'damu', 'Adivine'),
 (4, 2, 'martin', 'Sí! Cómo sabias?')]

In [112]:
# data.reduceByKey(lambda x: x[0]+y[0]).collect()

In [113]:
# data.groupByKey()

# 2016-01 Recuperatorio
Una red social almacena el contenido de los chats entre sus usuarios en un RDD que tiene registros con el siguiente formato: (chat_id, user_id, nickname, text). Queremos saber cuál es el usuario (user_id) que mas preguntas hace en sus chats, contabilizamos una pregunta por cada caracter “?” que aparezca en el campo text. Programar en Spark un programa que identifique al usuario preguntón.

In [114]:
# Con el map, lo llevo a algo de la forma (k,v) donde k es el id de usuario y v es la cantidad de veces de ?
d = data.map(lambda x: (x[1], x[3].count('?')))\
    #.reduceByKey(lambda x, y: x+y)\
    #.reduce(lambda x, y: x if (x[1] > y[1]) else y)

In [115]:
# Luego, reduzco por key, junto todas key sumandole sus counts. Deberia quedarme [(1,3),(2,1),(3,0)]
d = d.reduceByKey(lambda x,y: x+y)

In [116]:
# Me quedo con el mas grande en count. Nose si esto es lo mas efectivo, o deberia hacer un takeordered quizas.
pregunton = d.reduce(lambda x,y: x if x[1]>y[1] else y)

# 2016-01 Parcial
UBER almacena en un cluster todos los datos sobre el movimiento y viajes de todos sus vehículos. Existe un proceso que nos devuelve un RDD llamado trip_summary con los siguientes campos: (driver_id, car_id, trip_id, customer_id, date (YYYYMMDD), distance_traveled), Programar usando Py Spark un programa que nos indique cual fue el conductor con mayor promedio de distancia recorrida por viaje para Abril de 2016.



In [117]:
trips = [
    (1, 1, 1, 1, '20160101', 10),
    (2, 2, 2, 2, '20160202', 20),
    (1, 1, 3, 1, '20160402', 15),
    (1, 1, 4, 3, '20160405', 20),
    (2, 2, 5, 4, '20160410', 25),
    (3, 3, 6, 3, '20160415', 15),
    (2, 2, 7, 1, '20160420', 40),
    (3, 3, 8, 2, '20160505', 80)
]

In [118]:
data = sc.parallelize(trips)

In [119]:
# con el filter me quedo solo los de abril
data.filter(lambda x: x[4]>='20160401' and x[4]<'20160501')\
    .map(lambda x: (x[0], (1, x[5]) ))\
    .reduceByKey(lambda x,y: (x[0]+y[0], x[1] + y[1]) )\
    .reduce(lambda x,y: x if x[1][1]/x[1][0] > y[1][1]/y[1][0] else y   )

(2, (2, 65))

# 2015-02 2do Recuperatorio
Un telescopio registra automaticamente la luminosidad de distintas estrellas generando un RDD con registros de tipo (star_id, magnitude,spectral_type, timestamp). Queremos obtener un listado de estrellas que tienen el mismo tipo espectral e igual promedio de magnitud una vez redondeado el mismo a un decimal. El resultado debe ser una lista en donde cada elemento de la lista es una lista de ids de estrellas similares.

In [120]:
stars = [
    (1, 5, 1, 1),
    (2, 10, 1, 1),
    (3, 6, 1, 1),
    (4, 5.5, 2, 1),
    (1, 6, 1, 2),
    (2, 9, 1, 2),
    (3, 5, 1, 2),
    (1, 5.5, 1, 3),
    (2, 11, 1, 3),
    (3, 5.5, 1, 3)
]

In [156]:
data = sc.parallelize(stars)

In [162]:
data.map(lambda x: (x[0],(x[1],x[2], 1)))\
    .reduceByKey(lambda x,y: (x[0]+y[0], x[1], x[2]+y[2]))\
    .map(lambda x: ((x[1][1], x[1][0]/x[1][2]) ,x[0]))\
    .groupByKey()\
    .map(lambda x: (x[0], x[1].mean()))\
    .collect()
    #.reduceByKey(lambda x,y: )

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 168.0 failed 1 times, most recent failure: Lost task 0.0 in stage 168.0 (TID 1191, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "c:\Users\gaston.parente\AppData\Local\Programs\Python\Python37-32\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\worker.py", line 377, in main
  File "c:\Users\gaston.parente\AppData\Local\Programs\Python\Python37-32\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\worker.py", line 372, in process
  File "c:\Users\gaston.parente\AppData\Local\Programs\Python\Python37-32\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\serializers.py", line 393, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "c:\Users\gaston.parente\AppData\Local\Programs\Python\Python37-3

In [123]:
data.map(lambda x: (x[0], (x[2], x[1], 1)))\
    .reduceByKey(lambda x, y: (x[0], x[1]+y[1], x[2]+y[2]))\
    .map(lambda x: ((x[1][0], x[1][1]/x[1][2]), x[0]))\
    .groupByKey()\
    .map(lambda x: (x[0], list(x[1])))\
    .collect()

[((1, 5.5), [1, 3]), ((1, 10.0), [2]), ((2, 5.5), [4])]

# 2016-02 Parcial
En este ejercicio queremos programar un sistema que recomiende textos a usuarios en base a sus gustos sobre ciertos términos (palabras).

Se cuenta con un RDD de textos de la forma (docId, texto) donde texto es un string de longitud variable.

Además contamos con un RDD que indica qué términos le gustan o no a cada usuario de la forma (userId, término, score) por ejemplo (23, “calesita”, -2).

Se pide programar en Spark un programa que calcule el score total de cada documento para cada usuario generando un RDD de la forma (userId, docId, score) en donde el score es simplemente la suma de los scores del usuario para los términos que aparecen en el documento.

Puede haber términos en los documentos para los cuales no exista score de algunos usuarios, en estos casos simplemente los consideramos neutros (score=0)

In [124]:
documents = [
    (1, 'pablo honey'),
    (2, 'the bends'),
    (3, 'ok computer'),
    (4, 'kid a'),
    (5, 'amnesiac'),
    (6, 'hail to the thief'),
    (7, 'in rainbows'),
    (8, 'the king of limbs'),
    (9, 'a moon shaped pool')
]

scores = [
    ('thom', 'pablo', 1),
    ('thom', 'honey', 1),
    ('martin', 'pablo', -1),
    ('martin', 'honey', -1),
    ('martin', 'ok', 30),
    ('martin', 'computer', 30)
]

In [125]:

documentsRDD = sc.parallelize(documents)
scoresRDD = sc.parallelize(scores)

In [126]:
documentsRDD.collect()

[(1, 'pablo honey'),
 (2, 'the bends'),
 (3, 'ok computer'),
 (4, 'kid a'),
 (5, 'amnesiac'),
 (6, 'hail to the thief'),
 (7, 'in rainbows'),
 (8, 'the king of limbs'),
 (9, 'a moon shaped pool')]

In [127]:
words = documentsRDD.flatMap( lambda a: [(word, a[0]) for word in a[1].split()] )
words.collect()

[('pablo', 1),
 ('honey', 1),
 ('the', 2),
 ('bends', 2),
 ('ok', 3),
 ('computer', 3),
 ('kid', 4),
 ('a', 4),
 ('amnesiac', 5),
 ('hail', 6),
 ('to', 6),
 ('the', 6),
 ('thief', 6),
 ('in', 7),
 ('rainbows', 7),
 ('the', 8),
 ('king', 8),
 ('of', 8),
 ('limbs', 8),
 ('a', 9),
 ('moon', 9),
 ('shaped', 9),
 ('pool', 9)]

In [128]:
scores_for_join = scoresRDD.map(lambda x: (x[1], (x[0],x[2])) )
scores_for_join.collect()

[('pablo', ('thom', 1)),
 ('honey', ('thom', 1)),
 ('pablo', ('martin', -1)),
 ('honey', ('martin', -1)),
 ('ok', ('martin', 30)),
 ('computer', ('martin', 30))]

In [129]:
join = words.join(scores_for_join)
join.collect()

[('honey', (1, ('thom', 1))),
 ('honey', (1, ('martin', -1))),
 ('ok', (3, ('martin', 30))),
 ('pablo', (1, ('thom', 1))),
 ('pablo', (1, ('martin', -1))),
 ('computer', (3, ('martin', 30)))]

In [130]:
a = join.map(lambda x: ((x[1][1][0], x[1][0]) , x[1][1][1]) )
a.collect()

[(('thom', 1), 1),
 (('martin', 1), -1),
 (('martin', 3), 30),
 (('thom', 1), 1),
 (('martin', 1), -1),
 (('martin', 3), 30)]

In [131]:
data = a.reduceByKey(lambda x,y: x+y)\
        .map(lambda x: (x[0][0],x[0][1],x[1]))\
        .collect()

In [132]:
data

[('thom', 1, 2), ('martin', 1, -2), ('martin', 3, 60)]

In [133]:
# Si quisiera leer de un archivo: sc.textFile("README.md") esto devuelve el rdd.

# 2017-01 Parcial
Se tiene información estadística de la temporada regular de todos los jugadores de la NBA en un RDD de tuplas con el siguiente formato: (id_jugador, nombre, promedio_puntos, promedio_asistencias, promedio_robos, promedio_bloqueos, promedio_rebotes, promedio_faltas).

Un analista de la cadena ESPN está trabajando con un RDD que corresponde a la primera ronda de playoffs y que tiene el siguiente formato: (id_jugador, id_partido, timestamp, cantidad_puntos, cantidad_rebotes, cantidad_bloqueos, cantidad_robos, cantidad_asistencias, cantidad_faltas).

En base a estos RDDs se quiere programar en PySpark un programa que genere un RDD con los nombres (sin duplicados) de los jugadores que lograron en algún partido de playoffs una cantidad de asistencias mayor a su promedio histórico.

In [134]:
# usamos para simplificar el formato, que puede obtenerse con un map.
# (id_jugador, nombre, promedio_asistencias)
players_all_time_stats = [
    (1, 'Manu Ginobili', 800),
    (2, 'Kobe Bryant', 100),
    (3, 'Marc Gasol', 25),
    (4, 'James Harden', 1000)]

# usamos para simplificar el formato, que puede obtenerse con un map.
# (id_jugador, id_partido, timestamp, cantidad_asistencias)
scores = [
  (1, 1, 1, 100),
  (1, 1, 3, 100),
  (2, 1, 1, 150),
  (2, 1, 3, 150),
  (3, 2, 2, 50),
  (3, 2, 3, 50),      
  (1, 2, 1, 150),
  (1, 2, 3, 150),
]

In [135]:
jugadores = sc.parallelize(players_all_time_stats)
scores = sc.parallelize(scores)

In [136]:
scores_by_match = scores.map(lambda x: ((x[0],x[1]),x[3]))
scores_by_match.collect()

[((1, 1), 100),
 ((1, 1), 100),
 ((2, 1), 150),
 ((2, 1), 150),
 ((3, 2), 50),
 ((3, 2), 50),
 ((1, 2), 150),
 ((1, 2), 150)]

In [137]:
total_score_x_partido = scores_by_match.reduceByKey(lambda x,y: x+y)
total_score_x_partido.collect()

[((1, 2), 300), ((3, 2), 100), ((1, 1), 200), ((2, 1), 300)]

In [138]:
scores_by_match = total_score_x_partido.map(lambda x: (x[0][0], x[1]))
max_score_by_match = scores_by_match.reduceByKey(max)
max_score_by_match.collect()
#max_score_by_match.collect()

[(1, 300), (2, 300), (3, 100)]

In [139]:
jugadores = jugadores.map(lambda x: (x[0], (x[1],x[2])))
joined = max_score_by_match.join(jugadores)
joined.collect()


[(1, (300, ('Manu Ginobili', 800))),
 (2, (300, ('Kobe Bryant', 100))),
 (3, (100, ('Marc Gasol', 25)))]

In [140]:
result = joined.filter(lambda x: x[1][0]>=x[1][1][1])
result.collect()

[(2, (300, ('Kobe Bryant', 100))), (3, (100, ('Marc Gasol', 25)))]

In [141]:
result.map(lambda x: x[1][1][0] ).collect()

['Kobe Bryant', 'Marc Gasol']

# 2017_2C_1
Se cuenta con un RDD con información sobre patentamientos
de autos con la siguiente información (patente, marca, modelo,
versión, tipo_vehiculo, provincia, fecha), donde tipo_vehiculo
indica si la unidad patentada es auto, pickup, camión o moto.
Se pide generar un programa en pySpark que indique la marca y
modelo del auto más patentado por tipo de vehículo en la provincia
de Buenos Aires en el mes de Abril de 2017. (***) (15 pts)

In [142]:
datos_patentamientos = [
    ('MHG 100', 'Fiat', 'Siena', 1, 'auto', 'Buenos Aires', '2017-03-15'),
    ('MHG 101', 'Ford', 'Cargo 712', 2, 'camion', 'Chaco', '2017-03-19'),
    ('MHG 102', 'Ford', 'Cargo 712', 4, 'camion', 'Buenos Aires', '2017-04-01'),
    ('MHG 103', 'Fiat', 'Siena', 2, 'auto', 'Buenos Aires', '2017-04-02'),
    ('MHG 104', 'Chevrolet', 'Sonic', 1, 'auto', 'Buenos Aires', '2017-04-02'),
    ('MHG 105', 'Fiat', 'Siena', 3, 'auto', 'Uruguay', '2017-04-03'),
    ('MHG 106', 'Fiat', 'Siena', 1, 'auto', 'Buenos Aires', '2017-04-05'),
    ('MHG 107', 'Chevrolet', 'Sonic', 2, 'auto', 'Buenos Aires', '2017-04-17'),
    ('MHG 108', 'Chevrolet', 'Sonic', 1, 'auto', 'Buenos Aires', '2017-04-19'),
    ('MHG 109', 'Ford', 'Cargo 712', 4, 'camion', 'Buenos Aires', '2017-04-19'),
    ('MHG 110', 'Ford', 'Cargo 712', 2, 'camion', 'Buenos Aires', '2017-04-19'),
    ('MHG 111', 'Fiat', 'Siena', 3, 'auto', 'Cordoba', '2017-04-20'),
    ('MHG 112', 'Chevrolet', 'Sonic', 2, 'auto', 'Buenos Aires', '2017-04-21'),
    ('MHG 113', 'Fiat', 'Sedan', 2, 'auto', 'Buenos Aires', '2017-04-23'),
    ('MHG 114', 'Fiat', 'Sedan', 1, 'auto', 'Buenos Aires', '2017-04-24'),
    ('MHG 115', 'Honda', 'Hornet 160R', 1, 'moto', 'Buenos Aires', '2017-04-25'),
    ('MHG 116', 'Honda', 'Hornet 160R', 1, 'moto', 'Buenos Aires', '2017-04-25'),
    ('MHG 117', 'Ducati', 'SuperSport', 1, 'moto', 'Buenos Aires', '2017-04-26'),
    ('MHG 118', 'Scania', '420', 4, 'camion', 'Buenos Aires', '2017-04-26')
]

rdd_pt = sc.parallelize(datos_patentamientos)

In [143]:
patentamentos_abril = rdd_pt.filter(lambda x: x[6]>'2017-04-01' and x[6]<'2017-04-31')
patentamientos_bsas_en_abril = patentamentos_abril.filter(lambda x: x[5]=='Buenos Aires')

patentamientos_bsas_en_abril.map(lambda x: ( (x[1],x[2],x[4]), 1) )\
                            .reduceByKey(lambda x,y: x+y)\
                            .map(lambda x: (x[0][2], ((x[0][1], x[0][2]), x[1])))\
                            .reduceByKey(lambda x,y: x if x[1]>y[1] else y)\
                            .collect()
                                 

[('auto', (('Sonic', 'auto'), 4)),
 ('moto', (('Hornet 160R', 'moto'), 2)),
 ('camion', (('Cargo 712', 'camion'), 2))]

# 2018_1C_1
Nintendo of America (EEUU) tiene información de ventas de videojuegos físicas mensuales totalizadas en EEUU las cuales se realizan en cadenas de tiendas de videojuegos en el siguiente RDD: (id_videojuego, id_tienda, mes, anio, total_ventas_mensuales).

Por otro lado tenemos un RDD con información de las tiendas y de su ubicación (id_tienda, direccion, latitud, longitud, codigo_postal, estado).

Con esta información escribir un programa en pySpark para obtener la tienda que realizó menor cantidad de ventas en el estado de “Georgia” en todo el año 2017.

In [144]:
datos_ventas = [
    (0, 1, 2, 2017, 20),
    (2, 2, 3, 2017, 30),
    (1, 1, 7, 2017, 15),
    (0, 1, 3, 2015, 23),
    (3, 3, 1, 2017, 10),
    (3, 3, 8, 2016, 17),
    (3, 3, 6, 2017, 5),
    (0, 4, 10, 2017, 12),
    (15, 2, 5, 2017, 15),
    (0, 5, 1, 2017, 70)
]

datos_tiendas = [
    (1, 'Calle Falsa 123', -3, 4, 1834, 'Georgia'),
    (2, 'Siempre Viva 321', 3, -4, 4381, 'Georgia'),
    (3, 'Av Paseo Colon 850', 1, 2, 1063, 'Texas'),
    (4, 'Sarasa 120', 10, 1, 1234, 'California'),
    (5, 'Mostaza 564', 10, 3, 1324, 'Georgia'),
    (6, 'Ketchup 897', 3, -99, 7891, 'Los Angeles'),
    (4, 'Silver Rathalos 98287', 2, 1, 1234, 'Texas')
]

In [145]:
rdd_ventas = sc.parallelize(datos_ventas)
rdd_tiendas = sc.parallelize(datos_tiendas)

In [146]:
ventas_2017 = rdd_ventas.filter(lambda x: x[3]==2017)
tiendas_georgia = rdd_tiendas.filter(lambda x: x[5]=='Georgia')

In [147]:
tiendas_georgia = tiendas_georgia.map(lambda x: (x[0],x[5]))
ventas_2017 = ventas_2017.map(lambda x: (x[1], x[4]))

In [148]:
tiendas_georgia.collect()

[(1, 'Georgia'), (2, 'Georgia'), (5, 'Georgia')]

In [149]:
ventas_2017.collect()

[(1, 20), (2, 30), (1, 15), (3, 10), (3, 5), (4, 12), (2, 15), (5, 70)]

In [150]:
joined = ventas_2017.join(tiendas_georgia)

In [151]:
joined.collect()

[(1, (20, 'Georgia')),
 (1, (15, 'Georgia')),
 (2, (30, 'Georgia')),
 (2, (15, 'Georgia')),
 (5, (70, 'Georgia'))]

In [152]:
joined.map(lambda x: (x[0], x[1][0]))\
        .reduceByKey(lambda x,y: x+y)\
        .reduce(lambda x,y: x if x[1]<y[1] else y)\
        .collect()

AttributeError: 'tuple' object has no attribute 'collect'

# 2017_C2_2
A partir de la plataforma online (e-shop) de los países en los que opera, Nintendo tiene información de ventas de videojuegos diarias digitales por país en el siguiente RDD: (id_videojuego, codigo_pais, fecha, visitas_diarias, total_ventas_diarias).

Por otro lado se tienen otro RDD que tiene información de todos los videojuegos que se venden en su plataforma con el siguiente formato (id_videojuego, titulo, rating_pegi, rating_esbr). Tener en cuenta que un mismo videojuego se puede vender en distintos países y esos nos permitirá obtener métricas a nivel global.

Con esta información escribir un programa en pySpark que permita:

a) Obtener el videojuego con más ventas digitales globales (es decir en todos los países) en un RDD con el siguiente formato: (id_videojuego, titulo, total), siendo total la cantidad total de ventas digitales globales

b) Para el videojuego con mas ventas, obtener cual es el país para el cual ser registra una mayor tasa de conversión (es decir, mayor total_ventas_diarias / visitas_diarias)

In [188]:
datos_ventas_diarias = [
    (1, 'AR', '2018-01-01', 30, 10),
    (1, 'ES', '2018-01-01', 23, 13),
    (2, 'US', '2018-01-04', 45, 5),
    (2, 'MX', '2018-01-04', 20, 10),
    (2, 'US', '2018-01-06', 50, 15),
    (3, 'AR', '2018-01-06', 10, 2),
    (1, 'US', '2018-01-06', 14, 4),
    (3, 'ES', '2018-01-10', 34, 11),
    (4, 'ES', '2018-01-11', 42, 24),
    (4, 'US', '2018-01-11', 83, 34),
    (4, 'AR', '2018-01-11', 27, 20),
    (4, 'MX', '2018-01-11', 47, 18),
    (4, 'AR', '2018-01-20', 10, 0),
    (4, 'US', '2018-01-21', 34, 2),
    (4, 'ES', '2018-01-21', 25, 7)
]

datos_videojuegos = [
    (1, 'Zelda: Breath of the Wild', 9, 8),
    (2, 'Mario Kart', 9, 7),
    (3, 'Splatoon 2', 11, 8),
    (4, 'Monster Hunter Generations Ultimate', 13, 10)
]

In [196]:
rdd_ventas = sc.parallelize(datos_ventas_diarias)
rdd_videojuegos = sc.parallelize(datos_videojuegos)

In [197]:
rdd_ventas2 = rdd_ventas.map(lambda x: (x[0],x[4]))
rdd_videojuegos2 = rdd_videojuegos.map(lambda x: (x[0], x[1]))

In [198]:
# PUNTO A.
joined = rdd_ventas2.join(rdd_videojuegos2)

In [199]:
videojuego_mas_vtas = joined.map(lambda x: ((x[0],x[1][1]),x[1][0])) \
                            .reduceByKey(lambda x,y: x+y) \
                            .reduce(lambda x,y: x if x[1]>y[1] else y) \

id_vg = videojuego_mas_vtas[0][0]
id_vg

4

In [200]:
vg_mas_vtas = rdd_ventas.filter(lambda x: x[0]==id_vg)
vg_mas_vtas.collect()

[(4, 'ES', '2018-01-11', 42, 24),
 (4, 'US', '2018-01-11', 83, 34),
 (4, 'AR', '2018-01-11', 27, 20),
 (4, 'MX', '2018-01-11', 47, 18),
 (4, 'AR', '2018-01-20', 10, 0),
 (4, 'US', '2018-01-21', 34, 2),
 (4, 'ES', '2018-01-21', 25, 7)]

In [206]:
vg_mas_vtas.map(lambda x: (x[1],x[4]/x[3]))\
            .reduceByKey(lambda x,y: x if x>y else y)\
            .reduce(lambda x,y: x if x[1]>y[1] else y)

('AR', 0.7407407407407407)